Copyright (c) Meta Platforms, Inc. and affiliates.
This software may be used and distributed according to the terms of the Llama 2 Community License Agreement.

## Quick Start Notebook

This notebook shows how to train a Llama 2 model on a single GPU (e.g. A10 with 24GB) using int8 quantization and LoRA.

### Step 0: Install pre-requirements and convert checkpoint

The example uses the Hugging Face trainer and model which means that the checkpoint has to be converted from its original format into the dedicated Hugging Face format.
The conversion can be achieved by running the `convert_llama_weights_to_hf.py` script provided with the transformer package.
Given that the original checkpoint resides under `models/7B` we can install all requirements and convert the checkpoint with:

In [1]:
# %%bash
# pip install transformers datasets accelerate sentencepiece protobuf==3.20 py7zr scipy peft bitsandbytes fire torch_tb_profiler ipywidgets
# TRANSFORM=`python -c "import transformers;print('/'.join(transformers.__file__.split('/')[:-1])+'/models/llama/convert_llama_weights_to_hf.py')"`
# python ${TRANSFORM} --input_dir models --model_size 7B --output_dir models_hf/7B

### Step 1: Load the model

Point model_id to model weight folder

In [1]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer

model_id="../llama/hugging_face_weights/base/7B"

tokenizer = LlamaTokenizer.from_pretrained(model_id)

model =LlamaForCausalLM.from_pretrained(model_id, load_in_8bit=True, device_map='auto', torch_dtype=torch.float16)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('Asia/Tokyo')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/usr/local/cuda/lib64/libcudart.so'), PosixPath('/usr/local/cuda/lib64/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libc

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Step 2: Load the preprocessed dataset

We load and preprocess the samsum dataset which consists of curated pairs of dialogs and their summarization:

In [2]:
from pathlib import Path
import os
import sys
from utils.dataset_utils import get_preprocessed_dataset
from configs.datasets import samsum_dataset, receipt_dataset

train_dataset = get_preprocessed_dataset(tokenizer, receipt_dataset, 'train')


testinggg


In [3]:
len(train_dataset)

71

In [4]:
train_dataset.ann

[{'instruction': 'テキストから次の項目一覧の値をJson形式で出力してください。\n- 項目一覧\n[請求時分],[消費税額(8%)],[消費税額(10%)],[ページ番号],[支払者名],[請求者FAX],[支払通貨],[請求年月],[合計請求額(税抜)],[口座名義],[口座番号],[口座の種類],[銀行支店名],[銀行名],[支払期日],[消費税額],[合計請求額(税込)],[支払者会社名],[請求者電話番号],[請求者住所],[請求者会社名],[タイトル],[請求番号],[請求日付],[請求額(8%税込)],[請求額(10%税込)],[登録番号]',
  'input': '2023年 6月23日\t令和3年8月分\t前田道路株式会社 御中\n下記の通り請求致します\n請 求\t書\t(材料その他用)\n(業 者 控)\n適格請求書株式会社\n住\t所\n名\n発行者登録番号\tT1231231231235\n※支払期限\t2022/7/31\nみずほ\t銀行 東京\t支店\t普通\t当座\t1234567\n⑪\n(取引先コード欄)\n金額\n¥70,200\n月日\t品\t名\t納入場所\t工事 №.\t数量\t単位\t単価\t金\t額\t担当\n6/23\t*品名1\t三田倉庫\t001\t1.0\t個\t65,000\t65,000\n小\t計\t¥65,000\n10%\n消費税計\t消費税率\t8%\t¥5,200\n請求\t金\t額\t¥70,200\n(注)1.毎月末日締切で、翌月2日迄に必着するよう提出して下さい。\n2.提出用のシートを2枚印刷して、提出してください。\n3.取引先コード欄に貴社コードのゴム印を押印または、貴社コードを入力してください。\n',
  'output': '{\n"タイトル": "請 求 書",\n"請求日付": "2023年6月23日",\n"支払者会社名": "前田道路株式会社",\n"請求年月": "令和3年8月分",\n"支払通貨": "¥",\n"合計請求額(税込)": "(70,200",\n"合計請求額(税抜)": "65,000",\n"消費税額(8%)": "5,200",\n"請求者会社名": "適格請求書株式会社",\n"支払期日": "2022/7/31",\n"銀行

### Step 3: Check base model

Run the base model on an example input:

In [16]:
eval_prompt = """
以下のテキスト一覧は、pdfの請求書ドキュメントからOCRをした結果を左上から順番に並べたものです。テキストから次の項目一覧の値をJson形式で出力してください。存在しない項目に関しては出力しないでください。
### 項目一覧
[請求時分],[消費税額(8%)],[消費税額(10%)],[ページ番号],[支払者名],[請求者FAX],[支払通貨],[請求年月],[合計請求額(税抜)],[口座名義],[口座番号],[口座の種類],[銀行支店名],[銀行名],[支払期日],[消費税額],[合計請求額(税込)],[支払者会社名],[請求者電話番号],[請求者住所],[請求者会社名],[タイトル],[請求番号],[請求日付],[請求額(8%税込)],[請求額(10%税込)],[登録番号]
### テキスト一覧
2023年 6月23日\t令和3年8月分\t前田道路株式会社 御中
下記の通り請求致します
請 求\t書\t(材料その他用)
(業 者 控)
適格請求書株式会社
住\t所
名
発行者登録番号\tT1231231231235
※支払期限\t2022/7/31
みずほ\t銀行 東京\t支店\t普通\t当座\t1234567
⑪
(取引先コード欄)
金額
¥70,200
月日\t品\t名\t納入場所\t工事 №.\t数量\t単位\t単価\t金\t額\t担当
6/23\t*品名1\t三田倉庫\t001\t1.0\t個\t65,000\t65,000
小\t計\t¥65,000
10%
消費税計\t消費税率\t8%\t¥5,200
請求\t金\t額\t¥70,200
(注)1.毎月末日締切で、翌月2日迄に必着するよう提出して下さい。
2.提出用のシートを2枚印刷して、提出してください。
3.取引先コード欄に貴社コードのゴム印を押印または、貴社コードを入力してください。

### Json Output:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=500)[0], skip_special_tokens=True))


以下のテキスト一覧は、pdfの請求書ドキュメントからOCRをした結果を左上から順番に並べたものです。テキストから次の項目一覧の値をJson形式で出力してください。存在しない項目に関しては出力しないでください。
### 項目一覧
[請求時分],[消費税額(8%)],[消費税額(10%)],[ページ番号],[支払者名],[請求者FAX],[支払通貨],[請求年月],[合計請求額(税抜)],[口座名義],[口座番号],[口座の種類],[銀行支店名],[銀行名],[支払期日],[消費税額],[合計請求額(税込)],[支払者会社名],[請求者電話番号],[請求者住所],[請求者会社名],[タイトル],[請求番号],[請求日付],[請求額(8%税込)],[請求額(10%税込)],[登録番号]
### テキスト一覧
2023年 6月23日	令和3年8月分	前田道路株式会社 御中
下記の通り請求致します
請 求	書	(材料その他用)
(業 者 控)
適格請求書株式会社
住	所
名
発行者登録番号	T1231231231235
※支払期限	2022/7/31
みずほ	銀行 東京	支店	普通	当座	1234567
⑪
(取引先コード欄)
金額
¥70,200
月日	品	名	納入場所	工事 №.	数量	単位	単価	金	額	担当
6/23	*品名1	三田倉庫	001	1.0	個	65,000	65,000
小	計	¥65,000
10%
消費税計	消費税率	8%	¥5,200
請求	金	額	¥70,200
(注)1.毎月末日締切で、翌月2日迄に必着するよう提出して下さい。
2.提出用のシートを2枚印刷して、提出してください。
3.取引先コード欄に貴社コードのゴム印を押印または、貴社コードを入力してください。

### Json Output:
{
"請求額(10%税込)": "5,200",
"登録番号": "T1231231231235",
"請求番号": "TR231235",
"請求日付": "2023年 6月23日",
"請求者会社名": "適格請求書株式会社",
"請求者FAX": "054-345-6789",
"請求者電話番号": "054-345-6789",
"請求者住所": "〒543-0123 東京都港区三田 123-45-67",
"請求年月": "令和3年8月分

In [12]:
train_dataset[0]

{'input_ids': tensor([    1, 29871, 30651, 30557, 30199, 30572, 30454, 30255, 30279, 30287,
           235,   169,   170, 30449, 30330,  5140, 30199,   235,   174,   142,
         31376, 30854, 30335, 30454, 30645, 30604, 30203, 30279, 30412, 30513]),
 'labels': tensor([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100]),
 'attention_mask': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}

In [3]:
format = (
     "以下のテキスト一覧は、pdfの請求書ドキュメントからOCRをした結果を左上から順番に並べたものです。テキストから次の項目一覧の値をJson形式で出力してください。\n### 項目一覧\n[請求時分],[消費税額(8%)],[消費税額(10%)],[ページ番号],[支払者名],[請求者FAX],[支払通貨],[請求年月],[合計請求額(税抜)],[口座名義],[口座番号],[口座の種類],[銀行支店名],[銀行名],[支払期日],[消費税額],[合計請求額(税込)],[支払者会社名],[請求者電話番号],[請求者住所],[請求者会社名],[タイトル],[請求番号],[請求日付],[請求額(8%税込)],[請求額(10%税込)],[登録番号]"
     "### テキスト一覧\n{input}\n\n ### Json Output: \n "
)
index = 0
ann = train_dataset.ann[index]
prompt = format.format_map(ann)
example = prompt + ann["output"]


In [4]:
prompt = torch.tensor(
            train_dataset.tokenizer.encode(prompt), dtype=torch.int64
        )
example = train_dataset.tokenizer.encode(example)
example.append(train_dataset.tokenizer.eos_token_id)
example = torch.tensor(
    example, dtype=torch.int64
)
padding = train_dataset.max_words - example.shape[0]

In [5]:
padding = train_dataset.max_words - example.shape[0]

In [6]:
padding

786

We can see that the base model only repeats the conversation.

### Step 4: Prepare model for PEFT

Let's prepare the model for Parameter Efficient Fine Tuning (PEFT):

In [7]:
model.train()

def create_peft_config(model):
    from peft import (
        get_peft_model,
        LoraConfig,
        TaskType,
        prepare_model_for_int8_training,
    )

    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8,
        lora_alpha=32,
        lora_dropout=0.05,
        target_modules = ["q_proj", "v_proj"]
    )

    # prepare int-8 model for training
    model = prepare_model_for_int8_training(model)
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model, peft_config

# create peft config
model, lora_config = create_peft_config(model)



/usr/local/lib/python3.8/dist-packages/peft/utils/other.py:119: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


### Step 5: Define an optional profiler

In [13]:
from transformers import TrainerCallback
from contextlib import nullcontext
enable_profiler = False
output_dir = "tmp/llama-output"

config = {
    'lora_config': lora_config,
    'learning_rate': 1e-4,
    'num_train_epochs': 10,
    'gradient_accumulation_steps': 2,
    'per_device_train_batch_size': 2,
    'gradient_checkpointing': False,
}

# Set up profiler
if enable_profiler:
    wait, warmup, active, repeat = 1, 1, 2, 1
    total_steps = (wait + warmup + active) * (1 + repeat)
    schedule =  torch.profiler.schedule(wait=wait, warmup=warmup, active=active, repeat=repeat)
    profiler = torch.profiler.profile(
        schedule=schedule,
        on_trace_ready=torch.profiler.tensorboard_trace_handler(f"{output_dir}/logs/tensorboard"),
        record_shapes=True,
        profile_memory=True,
        with_stack=True)
    
    class ProfilerCallback(TrainerCallback):
        def __init__(self, profiler):
            self.profiler = profiler
            
        def on_step_end(self, *args, **kwargs):
            self.profiler.step()

    profiler_callback = ProfilerCallback(profiler)
else:
    profiler = nullcontext()

In [10]:
total_steps

NameError: name 'total_steps' is not defined

### Step 6: Fine tune the model

Here, we fine tune the model for a single epoch which takes a bit more than an hour on a A100.

In [14]:
from transformers import default_data_collator, Trainer, TrainingArguments



# Define training args
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    bf16=True,  # Use BF16 if available
    # logging strategies
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="no",
    optim="adamw_torch_fused",
    max_steps=total_steps if enable_profiler else -1,
    **{k:v for k,v in config.items() if k != 'lora_config'}
)

with profiler:
    # Create Trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        data_collator=default_data_collator,
        callbacks=[profiler_callback] if enable_profiler else [],
    )

    # Start training
    trainer.train()

Step,Training Loss
10,0.330000
20,0.245900
30,0.172100
40,0.168500
50,0.141300
60,0.139800
70,0.114900
80,0.123100
90,0.106800
100,0.091800


### Step 7:
Save model checkpoint

In [15]:
model.save_pretrained(output_dir)

### Step 8:
Try the fine tuned model on the same example again to see the learning progress:

In [17]:
model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=1000)[0], skip_special_tokens=True))



以下のテキスト一覧は、pdfの請求書ドキュメントからOCRをした結果を左上から順番に並べたものです。テキストから次の項目一覧の値をJson形式で出力してください。存在しない項目に関しては出力しないでください。
### 項目一覧
[請求時分],[消費税額(8%)],[消費税額(10%)],[ページ番号],[支払者名],[請求者FAX],[支払通貨],[請求年月],[合計請求額(税抜)],[口座名義],[口座番号],[口座の種類],[銀行支店名],[銀行名],[支払期日],[消費税額],[合計請求額(税込)],[支払者会社名],[請求者電話番号],[請求者住所],[請求者会社名],[タイトル],[請求番号],[請求日付],[請求額(8%税込)],[請求額(10%税込)],[登録番号]
### テキスト一覧
2023年 6月23日	令和3年8月分	前田道路株式会社 御中
下記の通り請求致します
請 求	書	(材料その他用)
(業 者 控)
適格請求書株式会社
住	所
名
発行者登録番号	T1231231231235
※支払期限	2022/7/31
みずほ	銀行 東京	支店	普通	当座	1234567
⑪
(取引先コード欄)
金額
¥70,200
月日	品	名	納入場所	工事 №.	数量	単位	単価	金	額	担当
6/23	*品名1	三田倉庫	001	1.0	個	65,000	65,000
小	計	¥65,000
10%
消費税計	消費税率	8%	¥5,200
請求	金	額	¥70,200
(注)1.毎月末日締切で、翌月2日迄に必着するよう提出して下さい。
2.提出用のシートを2枚印刷して、提出してください。
3.取引先コード欄に貴社コードのゴム印を押印または、貴社コードを入力してください。

### Json Output:
{
"請求額(10%税込)": "5,200",
"登録番号": "T1231231231235",
"請求番号": "TR231235",
"請求日付": "2023年 6月23日",
"請求者会社名": "適格請求書株式会社",
"請求者FAX": "054-345-6789",
"請求者電話番号": "054-345-6789",
"請求者住所": "〒543-0123 東京都港区三田 123-45-67",
"請求年月": "令和3年8月分

In [18]:
len(train_dataset)

71

In [ ]:
def get_eval_data():
    

In [ ]:
def calcluate_accuracy(model, dataset):
    